### Imports & Settings

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Load Data

In [9]:
# Cargar csv en un DataFrame
df = pd.read_csv("cars_data.csv")

# Explorar la estructura del DataFrame
df.info()
df.shape
df.isna().sum().sort_values(ascending=False)

# Ver el balance de las clases (84% Media, 10.5% Alta, 5.5% Baja)
df["calidad_auto"].value_counts(normalize=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   name                    10000 non-null  object 
 1   year                    10000 non-null  int64  
 2   selling_price           10000 non-null  int64  
 3   km_driven               10000 non-null  int64  
 4   fuel                    10000 non-null  int64  
 5   combustible_estimado_l  10000 non-null  float64
 6   seller_type             10000 non-null  int64  
 7   transmission            10000 non-null  int64  
 8   owner                   10000 non-null  int64  
 9   tipo_carroceria         10000 non-null  int64  
 10  potencia_motor_hp       10000 non-null  int64  
 11  nivel_seguridad         10000 non-null  float64
 12  calidad_auto            10000 non-null  object 
 13  score_calidad           10000 non-null  float64
 14  eficiencia_km_l         10000 non-null 

,proportion
calidad_auto,
Media,0.8399
Alta,0.1052
Baja,0.0549


#### Lo que esto nos dice
- Un fuerte desbalanceo de clases
  - "Media" es la mayoria con un 84%
  - "Alta" solo cubre el 10%, y "Baja" solo un 5.5%
- El accuracy de referencia (baseline) es engañosamente alto
  - Un DummyClassifier que siempre predice "Media" deberia de poder predecir 83.99%.
  - Superar este baseline no garantiza que el modelo sea mas precizo detectando "Alta" o "Baja".
- Riesgo de abandono de las clases minoritarias
  - El modelo tiene el riesgo de predecir "Media" la mayoria del tiempo debido al desbalanceo de las clases, lo que podria ignorar "Alta" o "Baja" como opciones.

### Cleaning



### Features

### Modeling

### Evaluation